In [2]:
import pandas as pd
import matplotlib 
import sklearn 
import sksurv
import numpy as np
import scipy.stats as stats
from scipy.stats import chisquare 

In [4]:
df = pd.read_csv("forecast_starting_data.csv")
df

,financial_active,net_payment_behaviour_tripd,promotion_flag,variable_rate_index,account_status_code,active_12_mths,bank_fico_buckets_20,charge_off_reason_code,mob,open_closed_flag,...,due_balance_2,due_balance_3,due_balance_4,due_balance_5,due_balance_6,due_balance_7,due_balance_8,snapshot,mth_code,industry
0,1.0,T,0,0,I 0-6,1.0,761+,NaN,42,1.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,B
1,0.0,P,0,0,A,1.0,761+,NaN,48,1.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,B
2,0.0,C,0,0,I 0-6,0.0,761+,NaN,244,1.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,A
3,1.0,R,0,0,I 0-6,1.0,621-640,NaN,76,1.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,A
4,1.0,P,0,0,I 0-6,1.0,761+,NaN,143,1.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19861,1.0,I,1,0,I 13-24,1.0,621-640,NaN,26,1.0,...,1363.04,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,A
19862,0.0,R,1,0,A,1.0,641-660,NaN,299,1.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,C
19863,1.0,I,0,0,I 0-6,1.0,741-760,NaN,4,1.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,C
19864,1.0,P,1,1,A,1.0,701-720,NaN,24,1.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,B


In [22]:
training_data = pd.read_csv("training_data.csv")

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (51) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [15]:
categorical = ["financial_active", "net_payment_behaviour_tripd", "promotion_flag", "account_status_code", "ever_delinquent_flg", "active_12_mths",
                "open_closed_flag", "purchase_active", "closed", "active", "charge_off", "charge_off_aged", "charge_off_bk", "writeoff_type_bko", 
                "writeoff_type_fraud_kiting", "writeoff_type_fraud_synthetic", "writeoff_type_deceased", "writeoff_type_other", "writeoff_type_aged",
                "writeoff_type_settlement", "writeoff_type_fraud_other", "writeoff_type_repo", "writeoff_type_null", "industry"]

categorical 

['financial_active',
 'net_payment_behaviour_tripd',
 'promotion_flag',
 'account_status_code',
 'ever_delinquent_flg',
 'active_12_mths',
 'open_closed_flag',
 'purchase_active',
 'closed',
 'active',
 'charge_off',
 'charge_off_aged',
 'charge_off_bk',
 'writeoff_type_bko',
 'writeoff_type_fraud_kiting',
 'writeoff_type_fraud_synthetic',
 'writeoff_type_deceased',
 'writeoff_type_other',
 'writeoff_type_aged',
 'writeoff_type_settlement',
 'writeoff_type_fraud_other',
 'writeoff_type_repo',
 'writeoff_type_null',
 'industry']

In [17]:
conting_table = []
for category in categorical:
    data_crosstab = pd.crosstab(training_data[category],
                    training_data['charge_off'],
                    margins=True, margins_name="Total")
    conting_table.append(data_crosstab)

len(conting_table)

24

In [67]:
data_crosstab = pd.crosstab(training_data['ever_delinquent_flg'],
                            training_data['charge_off'],
                           margins=True, margins_name="Total")

data_crosstab

charge_off,0,1,Total
ever_delinquent_flg,,,
0.0,5116718,13085,5129803
1.0,625807,2613,628420
Total,5742525,15698,5758223


In [62]:
test_list = [data_crosstab, data_crosstab]
for table in test_list:
    print(table)



charge_off                 0      1    Total
ever_delinquent_flg                         
0.0                  5116718  13085  5129803
1.0                   625807   2613   628420
Total                5742525  15698  5758223
charge_off                 0      1    Total
ever_delinquent_flg                         
0.0                  5116718  13085  5129803
1.0                   625807   2613   628420
Total                5742525  15698  5758223


In [37]:
chi_square = 0
rows = training_data['ever_delinquent_flg'].unique()
columns = training_data['charge_off'].unique()
for i in columns:
    for j in rows:
        O = data_crosstab[i][j]
        E = data_crosstab[i]['Total'] * data_crosstab['Total'][j] / data_crosstab['Total']['Total']
        chi_square += (O-E)**2/E

chi_square

531.9465241903131